<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/TopicModel_RecEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install and imports

In [1]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_lg
!pip install -U kaleido
!pip install bertopic
# !python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.1 MB/s 
     |████████████████████████████████| 457 kB 66.2 MB/s 
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 10.1 MB 35.7 MB/s 
     |████████████████████████████████| 181 kB 76.9 MB/s 
     |████████████████████████████████| 660 kB 53.6 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 7.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 156 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.3 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 5.2 MB 45.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 8.5 MB/s 
     |████████████████████████████████| 4.2 MB 47.9 MB/s 
     |████████████████████████████████| 1.2 MB 54.8 MB/s 
     |███████████

In [2]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido
nlp = spacy.load('en_core_web_lg')
multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2

## helper functions

In [27]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

def remove_punct_df(df):

  # remove punctuation
  # text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  filt_combined = []
  for word in word_tokenize(df):

    if word.lower() not in string.punctuation:
      # print(word)
      
      filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

def create_input_new(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']

  # remove blanks
  df = df[df['Combined']!=" "]

  #keep only selected cols
  df_sel = df[['Authors','Pub Year','Research Area','Combined',"Pub TYpe"]]
  df_sel= df_sel.rename(columns={'Pub Year':'pub_year',"Research Area":"research_area","Authors":"authors","Combined":"combined","Pub TYpe":"pub_type"})

  combined = list(df_sel['combined'])

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, '', l )
    comb_clean.append(mod_string)

  # merge back to df
  df_sel['combined'] = comb_clean

  # filter spurtious yeats
  df_sel = df_sel[df_sel['pub_year']!='12.0.1.2']

  # convert years to int
  # df_sel['pub_year'] = df_sel['pub_year'].astype(str)
  df_sel['pub_year'] = df_sel['pub_year'].astype(str).replace('\.0', '', regex=True).astype(int)
  # if year is 201, that is mistyped fom 2001
  df_sel[df_sel['pub_year']==201]['pub_year'] = 2001


  return df_sel

def lemma_spacy(df_combined):

  filt_combined = []
  for word in nlp(df_combined):
    if word.lemma_ != '-PRON-' :
      filt_combined.append(word.lemma_)

  new_df = " ".join(filt_combined)

  return new_df  

def remove_stop_df(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip



def flatten(t):
    return [item for sublist in t for item in sublist]


def get_authors(input_data,rep_docs):
  tt = input_data['combined'].to_list()

  auth_list= []
  for d in rep_docs:
    ind = tt.index(d)
    auth_str = input_data.authors[ind]
    auth_el = auth_str.split(" ,")
    auth_list.append(auth_el)

  auth_list = flatten(auth_list)
  if '' in auth_list:
    auth_list.remove('')

  # unduplicate repeating authors 
  # print("Length of List:",len(auth_list))
  # print(auth_list)


  final_auth_list = list(set(auth_list))
  if '' in final_auth_list:
    final_auth_list.remove('')

  # print(final_auth_list)
  # print("Length of Final List:",len(final_auth_list))

  return final_auth_list

  
def author_all_topics(model,input_data):

  dict_df = {}
  for i in range(len(model.topics)-1):
    rep_docs = model.get_representative_docs(i)
    author_list = get_authors(input_data,rep_docs)

    dict_df[i] = author_list

  # create df with topics and authors
  author_topics = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_df.items() ]))

  return author_topics,dict_df


# find closest author to each doc by using each doc as a search term

def closest_author_docs(relevant_docs):

  auth_list = []

  for doc in relevant_docs:
    similar_topics, similarity = topic_model.find_topics(doc, top_n=5)
    auth_list.append(dict_authors[similar_topics[0]])

  final_auth_list = list(set(flatten(auth_list)))


  return final_auth_list



## path of files 

In [4]:
drive.mount('/content/drive')
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

Mounted at /content/drive


['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

## main code

In [6]:
# which model to use - "default" or "specter"
model_use = "default"

# which beamline?
beamline = "8.3.2"



In [7]:
#####

start_time = time.time()
iter_version = "Topic_RecEngine"
base_path = '/content/drive/MyDrive/NLP/'

# import csv of beamline
beam_name = "Beamline_" + beamline + ".xls"
df = pd.read_table(path + beam_name, on_bad_lines='skip')


# combine abstract and title
input_data = create_input_new(df)

# remove stopwords
input_data['combined'] = input_data['combined'].apply(remove_stop_df)

# lemmatize
input_data['combined'] = input_data['combined'].apply(lemma_spacy)

# remove punct
input_data['combined'] = input_data['combined'].apply(remove_punct_df)

# train topic model
if model_use == "specter":
  sentence_model = SentenceTransformer('allenai-specter')
  topic_model = BERTopic(embedding_model=sentence_model,verbose=True) 
else: 
  topic_model = BERTopic(verbose=True) # uses default bertopic model - "all-miniLM-L6_v2"

list_text = input_data['combined'].to_list()

topics, probs = topic_model.fit_transform(list_text)

print('Topics:',topics)

# create beamline folder in the given path to which model files will be saved 
model_path = base_path + iter_version + beam_name + "/"
create_path_if_not_exists(model_path)

# save topic model as pickle file
file_path = model_path + "model" + beam_name + ".pkl"
joblib.dump(topic_model, file_path) 

# visualize barchart of topics
fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
fig_name = model_path + "bar_chart" + beam_name  +".html"
fig_name_png = model_path + "bar_chart" + beam_name  +".png"
fig.write_html(fig_name)
fig.write_image(fig_name_png)

# save topics in excel file
excel_name = model_path + "Topic_Results.xlsx"
df_topics = pd.DataFrame(topic_model.topics)
df_topics.to_excel(excel_name,sheet_name="topic_words")

## dynamic topic modeling - topics over time 
years = input_data['pub_year'].to_list() # save years from the dataframe
topics_over_time = topic_model.topics_over_time(list_text, topics, years) # train dynamic topic model
fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20) # save figure
fig_name_png = model_path + "topic_time" + beam_name  +".png"
fig_time.write_image(fig_name_png) # static image 
fig_name_html = model_path + "topic_time" + beam_name  +".html"
fig_time.write_html(fig_name_html) # interactive html image


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-05-24 15:23:05,677 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-24 15:23:14,704 - BERTopic - Reduced dimensionality
2022-05-24 15:23:14,735 - BERTopic - Clustered reduced embeddings


Topics: [-1, 2, 2, 2, 2, 2, 3, -1, 3, 2, 2, 0, -1, 0, 2, 0, 2, 2, 0, -1, 3, 0, 0, 2, 0, 4, 9, 0, 6, 5, 0, -1, 4, 4, 0, 2, 10, 10, 10, 10, 0, -1, 3, -1, 2, 4, 10, 2, 4, -1, 9, 4, 1, 2, 2, 2, 0, -1, -1, 0, 0, 10, 10, 0, 10, 2, 2, 0, -1, -1, -1, 3, -1, 2, 0, 0, 0, 0, 3, 1, -1, 2, -1, 4, 1, -1, 2, 4, 0, 8, 0, 0, 0, 2, 2, 2, 0, 3, 0, -1, 3, 3, 3, 3, 4, 4, 1, 1, 0, 0, 4, 0, 4, -1, 5, 3, 1, 1, 2, 1, 1, 2, -1, 3, -1, 8, -1, -1, -1, -1, -1, 5, 0, 0, -1, 8, 3, 3, 1, 6, 8, -1, 6, -1, -1, 3, 1, -1, -1, 2, 3, 0, 4, -1, 2, 6, 0, 0, 0, 0, 9, -1, -1, -1, -1, 5, 5, 8, 0, 7, 9, 9, -1, -1, -1, 0, 1, 1, 1, 0, 0, 0, -1, 1, 1, -1, 0, 2, 4, 7, 7, -1, 5, 3, 7, 2, 0, 1, 1, 5, 6, 6, 2, 8, 5, -1, 4, 3, 3, 3, 9, 1, 6, 0, 1, 2, 1, 3, 0, 4, 0, 2, 0, 0, 2, 5, 5, 1, 0, -1, 4, 1, 4, 4, 0, 7, 5, 9, 2, 7, 9, 5, 5, 9, -1, 1, 0, 3, -1, 7, 6, 5, 3, 0, -1, 6, 9, 9, 7, 7, 0, 0, 6, 6, 3, 1, 1, 1, 1, 3, 1, 9, 3, 0, -1, 3, 0, 5, -1, 8, 8, 8, -1, 1, -1, 4, 0, 2, 1, -1, 0, 0, 0, 7, -1, 2, -1, 1, 2, 0, 9, 0, -1, 4, 5, 2, 2, -1, -1

19it [00:00, 41.06it/s]


In [10]:
##### Rec Engine


###--------- for a given search term, find closest authors, documents-----------###
search_term = input("Find authors and documents related to the term : ")


# generating authors for each topic 
df_authors, dict_authors = author_all_topics(topic_model,input_data)
# save df with authors 
excel_name = model_path + "Author_Topics.xlsx"
df_authors.to_excel(excel_name,sheet_name="authors_topics")

# find most similar topics
similar_topics, similarity = topic_model.find_topics(search_term, top_n=5)

# find all authors in the most similar topic
print("Authors are:",dict_authors[similar_topics[0]])

# get all docs related to a particular search term 
rep_docs = topic_model.get_representative_docs(similar_topics[0])
print("Closest documents are:")
rep_docs


Find authors and documents related to the term : battery
Authors are: ['Antypas, K.', 'Parkinson, Dilworth Y.,Yuan, Rodger', 'Ramakrishnan, L.', 'Nugent, P.', 'Gerhardt, L.', 'Tull, C.', 'Jacobsen, D.', 'Monga, I.', 'Snavely, C.', 'Dart, E.', 'Fagnan, K.', 'Hallinan, D.T.', 'Parkinson, D.', 'MacDowell, Alastair A.,Balsara, Nitash P.,', 'Hexemer, A.', 'Canon, S.', 'Devaux, Didier H,Harry, Katherine', "O'Brien, M.", 'Lockwood, G.K.']
Closest documents are:


['lithium Sulfur Batteries Block Copolymer Electrolyte analyze x ray Microtomography work lithium sulfur li battery use liquid electrolyte limit stability couple Li metal anode study li battery solid block copolymer electrolyte exhibit improve stability Li anode electrolyte comprise polystyrene b poly ethylene oxide seo copolymer dope Li salt hollow carbon nanosphere impregnate sulfur use build composite cathode two type sulfur impregnate functionalize carbon nanosphere use one carboxylic acid group short lithium poly 4 styrenesulfonyl trifluoromethylsulfonyl imide PSTFSI Li chain cell Li2S8 dissolve seo base electrolyte serve baseline cycling reason capacity fade determine imaging battery use synchrotron hard x ray microtomography generally assume lis cell fail due dissolution polysulfide liquid electrolyte i.e main problem relate cathode all solid cell failure primarily due delamination Li foil polymer electrolyte layer delamination also observe sulfur cathode likely large change vol

In [11]:
###----- for a given document, find closest authors and documents-----------###


search_doc = "Researchers have increased the lifetime of a promising electric vehicle \\
 battery to a record level, an important step toward the goal of lighter, less expensive and long-lasting batteries for future electric vehicles. The work is reported June 28 in the journal Nature Energy.Such batteries—the goal\\
  of research groups the world over—are seen as an important part of the solution to reduce the effects of climate change, and scientists are exploring a dizzying array of options.\\
One solution on the horizon is a lithium-metal battery for electric vehicles. These batteries hold almost twice the energy of their widely used lithium-ion counterparts, and they’re lighter.\\
 That combination offers the enticing prospect of an electric vehicle that would be lighter and go much farther on a single charge. But lithium-metal batteries in the laboratory have been plagued by premature death, lasting only a fraction of the time of today’s lithium-ion batteries."



# find most similar topic
similar_topics, similarity = topic_model.find_topics(search_doc, top_n=5)

# find all authors in the most similar topic
print("Authors are:",dict_authors[similar_topics[0]])

# get all docs related to a particular search term 
rep_docs = topic_model.get_representative_docs(similar_topics[0])
print("Closest documents are:")
rep_docs


Authors are: ['Antypas, K.', 'Parkinson, Dilworth Y.,Yuan, Rodger', 'Ramakrishnan, L.', 'Nugent, P.', 'Gerhardt, L.', 'Tull, C.', 'Jacobsen, D.', 'Monga, I.', 'Snavely, C.', 'Dart, E.', 'Fagnan, K.', 'Hallinan, D.T.', 'Parkinson, D.', 'MacDowell, Alastair A.,Balsara, Nitash P.,', 'Hexemer, A.', 'Canon, S.', 'Devaux, Didier H,Harry, Katherine', "O'Brien, M.", 'Lockwood, G.K.']
Closest documents are:


['lithium Sulfur Batteries Block Copolymer Electrolyte analyze x ray Microtomography work lithium sulfur li battery use liquid electrolyte limit stability couple Li metal anode study li battery solid block copolymer electrolyte exhibit improve stability Li anode electrolyte comprise polystyrene b poly ethylene oxide seo copolymer dope Li salt hollow carbon nanosphere impregnate sulfur use build composite cathode two type sulfur impregnate functionalize carbon nanosphere use one carboxylic acid group short lithium poly 4 styrenesulfonyl trifluoromethylsulfonyl imide PSTFSI Li chain cell Li2S8 dissolve seo base electrolyte serve baseline cycling reason capacity fade determine imaging battery use synchrotron hard x ray microtomography generally assume lis cell fail due dissolution polysulfide liquid electrolyte i.e main problem relate cathode all solid cell failure primarily due delamination Li foil polymer electrolyte layer delamination also observe sulfur cathode likely large change vol

In [38]:
###----- for a given auhors, find all other authors and closest documents-----------###

# search_author = "Manga, M."

# iterate thru dict, pick up the topic where that author occurs and return all authors in that topic
search_author = 'Ramakrishnan, L.'
for t in dict_authors:
  if search_author in dict_authors[t]:
    print("Topic Number is:",t)
    first_auth_list = dict_authors[t]
    print("All other authors:",dict_authors[t])


# find the doc in which the search author occurs, then find closest docs to that doc and return authors for that
relevant_docs = input_data[input_data.apply(lambda row: row.astype(str).str.contains(search_author).any(), axis=1)]['combined'].to_list()
authors_docs = closest_author_docs(relevant_docs)
print("Authors representing the closest docs are:",authors_docs)

final_auth_list = authors_docs + first_auth_list
final_auth_list = list(set(final_auth_list))
print("Final Author list:",final_auth_list)



Topic Number is: 6
All other authors: ['Antypas, K.', 'Parkinson, Dilworth Y.,Yuan, Rodger', 'Ramakrishnan, L.', 'Nugent, P.', 'Gerhardt, L.', 'Tull, C.', 'Jacobsen, D.', 'Monga, I.', 'Snavely, C.', 'Dart, E.', 'Fagnan, K.', 'Hallinan, D.T.', 'Parkinson, D.', 'MacDowell, Alastair A.,Balsara, Nitash P.,', 'Hexemer, A.', 'Canon, S.', 'Devaux, Didier H,Harry, Katherine', "O'Brien, M.", 'Lockwood, G.K.']
Topic Number is: 7
All other authors: ['Tull, Craig E,', 'Ramakrishnan, L.', 'Vogelgesang, M.', 'Hexemer, Alexander', 'Sethian, J.', 'Blair, J.', 'Tamura, Nobumichi', 'Loo, W.S.', 'Faragó, T.', 'MacDowell, Alastair A.,Parkinson, Dilworth Y.,Patton, S.J.', 'Maslyn, Jacqueline', 'Parkinson, D.Y.', 'Tierney, B.L.', 'Basham, M.', 'Wadeson, N.', 'Parkinson, Dilworth Y.,Balsara, Nitash P.,', 'Canon, R.S.', 'Archibald, R.', 'Bicer, T.', 'McEntush, Kyle', 'Mohan, K.A.', 'Essiari, A.', 'Gursoy, D.', 'Venkatakrishnan, S.', 'Harry, Katherine', 'Oh, H.J.', 'Deslippe, J.', 'Pelt, D.M.']
Authors represe

In [39]:
relevant_docs

['high performance data management analysis tomography Advanced Light Source ALS third generation synchrotron x ray source operate user facility 40 beamline host 2000 user per year around world Users Hard X ray Micro Tomography Beamline 8.3.2 often collect 1 Terabyte raw datum per day turn generate additional Terabytes process datum datum rate continue increase rapidly due fast detector new sample automation capability present development deployment computational pipeline fed data ALS powered storage networking compute resource local National Energy Research Scientific Computing Center NERSC Energy Sciences Network ESNET one year operation system contain 70,000 dataset 350 TB datum 85 user dataset collect Hard X ray Tomography Beamline automatically reconstruct use parameter set user and/or automatically detect datum acquisition control system result present user visualization secure web portal user download datum launch currently limited grow number operation base datum such filtering